In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

from myfm import MyFMRegressor
from myfm import MyFMOrderedProbit
from myfm.utils.callbacks import RegressionCallback

class MyRegressionCallback(RegressionCallback):
    def __call__(self, i, fm, hyper, history):
        should_stop, description = super(MyRegressionCallback, self).__call__(i, fm, hyper, history)
        trace_result = self.result_trace[-1]
        if len(self.result_trace) > 8:
            for index in range(8):
                old_trace_result = self.result_trace[-(index + 1)]
                if abs(old_trace_result['rmse'] - trace_result['rmse']) > 0.0001:
                    return (should_stop, description)
            return (True, description)
        return (should_stop, description)

In [ ]:
path = 'dataset/preprocessed/'
data = pd.read_csv(path + 'ml-100k-preprocessed.csv', sep=',', encoding='latin-1', nrows=10000, index_col=None)

In [ ]:
import json

In [ ]:
with open(path + 'ml-100k-preprocessed.json') as f:
    meta_data = json.load(f)
    group_shapes = [v for k,v in meta_data.items()]

In [ ]:
group_shapes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
  data.drop(['rating', 'timestamp'], axis = 1), data['rating'], random_state=42
)

In [ ]:
group_shapes

In [33]:
X_train.shape

(7500, 2696)

In [ ]:
callback = MyRegressionCallback(5, X_test, y_test.values)

reg = MyFMRegressor(rank=1).fit(X_train, y_train, n_iter=400, group_shapes=group_shapes, callback=callback)
ord = MyFMOrderedProbit(rank=0).fit(X_train, y_train, n_iter=400, group_shapes=group_shapes)
rf = RandomForestRegressor(n_estimators = 400, random_state = 42).fit(X_train, y_train)

reg_error = metrics.mean_squared_error(y_test, reg.predict(X_test), squared=False)
print(f'FM Regression error: {reg_error}')

# ord_error = metrics.mean_squared_error(y_test, ord.predict(X_test), squared=False)
# print(f'FM Ordinal Regression error: {ord_error}')

# rf_error = metrics.mean_squared_error(y_test, rf.predict(X_test), squared=False)
# print(f'Random Forest error: {rf_error}')

alpha=0.0000, rmse_mean=447340271593.1005, rmse_this=126288164.0527, rmse_all_but_5=126217158.5504: 100%|█| 400/400 [00
